In [1]:
import pandas as pd
import numpy as np
IRI_Selected_18=['LOS ANGELES','DALLAS/FT WORTH','NEW YORK','HOUSTON','WEST TEX/NEW MEXICO','PHOENIX/TUCSON','SOUTH CAROLINA',
                'MIAMI/FT LAUDERDALE','BALTIMORE/WASHINGTON','BIRMINGHAM/MONTGOMERY','NEW ORLEANS/MOBILE',
                 'CHICAGO','MISSISSIPPI','BOSTON','ATLANTA','DETROIT','RALEIGH/GREENSBORO','DENVER']

# Target Audience

In [2]:
teen_zips=pd.read_excel("/home/jian/SunnyD/Otherinput/2016/ACS_16_5YR_DP05_with_ann (Demographic Population).xlsx",skiprows=1,dtype=str)
teen_zips=teen_zips[['Id','Estimate; SEX AND AGE - 10 to 14 years','Estimate; SEX AND AGE - 15 to 19 years','Estimate; SEX AND AGE - Total population']]
teen_zips=teen_zips.rename(columns={"Estimate; SEX AND AGE - Total population":"Total_Population"})
teen_zips['zcta']=teen_zips['Id'].apply(lambda x: x.split("US")[1])
teen_zips['Estimate; SEX AND AGE - 10 to 14 years']=teen_zips['Estimate; SEX AND AGE - 10 to 14 years'].astype(float)
teen_zips['Estimate; SEX AND AGE - 15 to 19 years']=teen_zips['Estimate; SEX AND AGE - 15 to 19 years'].astype(float)
teen_zips['Total_Population']=teen_zips['Total_Population'].astype(float)
teen_zips['teen_population']=teen_zips['Estimate; SEX AND AGE - 10 to 14 years']+teen_zips['Estimate; SEX AND AGE - 15 to 19 years']
teen_zips=teen_zips[['zcta','teen_population','Total_Population']]

In [3]:
female_zips=pd.read_csv("/home/jian/SunnyD/Otherinput/2016/ACS_16_5YR_B01001_with_ann (Age by Sex).csv",skiprows=1,dtype=str)
female_zips=female_zips[['Id','Estimate; Female: - 25 to 29 years','Estimate; Female: - 30 to 34 years','Estimate; Female: - 35 to 39 years',
                         'Estimate; Female: - 40 to 44 years','Estimate; Female: - 45 to 49 years','Estimate; Female: - 50 to 54 years']]
female_zips.columns=['Id','Female_25_29','Female_30_34','Female_35_39','Female_40_44','Female_45_49','Female_50_54']

female_age=['Female_25_29','Female_30_34','Female_35_39','Female_40_44','Female_45_49','Female_50_54']
for col in female_age:
    female_zips[col]=female_zips[col].astype(float)
female_zips['Female_25_54']=female_zips['Female_25_29']+female_zips['Female_30_34']+female_zips['Female_35_39']+\
                                female_zips['Female_40_44']+female_zips['Female_45_49']+female_zips['Female_50_54']
female_zips['zcta']=female_zips['Id'].apply(lambda x: x.split("US")[1])
female_zips=female_zips[['zcta','Female_25_54']]

In [4]:
household=pd.read_csv("/home/jian/SunnyD/Otherinput/2016/ACS_16_5YR_S1101_with_ann (Household Composition).csv",dtype=str,skiprows=1)
household=household[['Id','Total; Estimate; Total households','Total; Estimate; AGE OF OWN CHILDREN - Households with own children of the householder under 18 years']]
household.columns=['Id','Total_HH','Total_HH_Child']
household['Total_HH_Child']=household['Total_HH_Child'].astype(float)
household['Total_HH']=household['Total_HH'].astype(float)
household['HH_With_Child_Pctg']=household['Total_HH_Child']/household['Total_HH']
household['zcta']=household['Id'].apply(lambda x: x.split("US")[1])
household=household[['zcta','HH_With_Child_Pctg','Total_HH']]

In [5]:
Income_HH=pd.read_csv("/home/jian/SunnyD/Otherinput/2016/ACS_16_5YR_DP03_with_ann (Demo work force).csv",dtype=str,skiprows=1)
Income_HH=Income_HH[['Id','Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - Less than $10,000',
                    'Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - $10,000 to $14,999',
                    'Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - $15,000 to $24,999',
                    'Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - $25,000 to $34,999',
                    'Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - $35,000 to $49,999']]
Income_HH.columns=['Id','HH_Income_Lower_10K','HH_Income_10K_15K','HH_Income_15K_25K','HH_Income_25K_35K','HH_Income_35K_50K']
Income_Col=['HH_Income_Lower_10K','HH_Income_10K_15K','HH_Income_15K_25K','HH_Income_25K_35K','HH_Income_35K_50K']
for col in Income_Col:
    Income_HH[col]=Income_HH[col].astype(float)
Income_HH['HH_Income_0_50K']=Income_HH['HH_Income_Lower_10K']+Income_HH['HH_Income_10K_15K']+Income_HH['HH_Income_15K_25K']+\
                                Income_HH['HH_Income_25K_35K']+Income_HH['HH_Income_35K_50K']
Income_HH['zcta']=Income_HH['Id'].apply(lambda x: x.split("US")[1])
Income_HH=Income_HH[['zcta','HH_Income_0_50K']]

In [6]:
Combined_Demo=pd.merge(teen_zips,female_zips,on='zcta',how='outer')
Combined_Demo=pd.merge(Combined_Demo,household,on='zcta',how='outer')
Combined_Demo=pd.merge(Combined_Demo,Income_HH,on='zcta',how='outer')
Combined_Demo['Monther']=Combined_Demo['Female_25_54']*Combined_Demo['HH_With_Child_Pctg']
Combined_Demo['Monther']=Combined_Demo['Monther'].apply(lambda x:round(x,2))

In [7]:
writer=pd.ExcelWriter("/home/jian/SunnyD/Primary_DMA_Audience/SunnyD Target audience penetration_JL_20180501.xlsx",engine='xlsxwriter')
Combined_Demo.to_excel(writer,'demo_data',index=False)

Combined_Demo=Combined_Demo[['zcta','teen_population','Monther','HH_Income_0_50K','Total_Population','Total_HH']]

# Zip Code to ZCTA

In [8]:
match_zip=pd.read_excel("/home/jian/SunnyD/Otherinput/2016/zip_to_zcta_2017_From udsmapper.xlsx",dtype=str)
match_zip=match_zip[['ZIP_CODE','ZCTA']]
match_zip.columns=['zip_cd','zcta']

In [9]:
Zip_IRI=pd.read_excel("/home/jian/SunnyD/Otherinput/All_IRI_Mkt_with_Zips_0430.xlsx",dtype=str)
Zip_IRI=pd.merge(Zip_IRI,match_zip,on='zip_cd',how='left')
Zip_IRI=Zip_IRI[['zcta','IRI Market']].drop_duplicates()
Zip_IRI=Zip_IRI[~pd.isnull(Zip_IRI['zcta'])].reset_index()
del Zip_IRI['index']
Zip_IRI_NoMarket=Zip_IRI[Zip_IRI['IRI Market']=="NO MARKET"]
Zip_IRI_64Market=Zip_IRI[Zip_IRI['IRI Market']!="NO MARKET"]
Zip_IRI_NoMarket=Zip_IRI_NoMarket[~Zip_IRI_NoMarket['zcta'].isin(Zip_IRI_64Market['zcta'])]
Zip_IRI_18_Markets=Zip_IRI[Zip_IRI['IRI Market'].isin(IRI_Selected_18)].drop_duplicates()
Zip_IRI_48_Markets=Zip_IRI_64Market[~Zip_IRI_64Market['zcta'].isin(Zip_IRI_18_Markets['zcta'])].drop_duplicates()
Zip_IRI_18_Markets.reset_index(inplace=True)
del Zip_IRI_18_Markets['index']

In [10]:
Zip_DMA=pd.read_excel("/home/jian/Docs/Geo mapping/Zips by DMA by County16-17 nielsen.xlsx",dtype=str,skiprows=1)
Zip_DMA=Zip_DMA.iloc[:,[0,2]]
Zip_DMA.columns=['zip_cd','DMA']
Zip_DMA=pd.merge(Zip_DMA,match_zip,on='zip_cd',how='left')
Zip_DMA=Zip_DMA[['zcta','DMA']].drop_duplicates()
Zip_DMA=Zip_DMA[~pd.isnull(Zip_DMA['zcta'])].reset_index()
del Zip_DMA['index']

In [11]:
Zip_IRI_18_Markets=pd.merge(Zip_IRI_18_Markets,Combined_Demo,on="zcta",how='left')
Zip_DMA=pd.merge(Zip_DMA,Combined_Demo,on="zcta",how='left')

In [12]:
Zip_IRI_18_Markets.to_excel(writer,"IRI_18_Market_Demo_by_zcta",index=False)
Zip_DMA.to_excel(writer,"DMA_Demo_by_zcta",index=False)

In [13]:
Zip_DMA_Overall=Zip_DMA.groupby(['DMA'])['teen_population','Monther','HH_Income_0_50K','Total_Population','Total_HH'].sum()
Zip_DMA_Overall.reset_index(inplace=True)
Zip_DMA_Overall['Total_Target_Audience']=Zip_DMA_Overall['teen_population']+Zip_DMA_Overall['Monther']
Zip_DMA_Overall_Zip=Zip_DMA[['DMA','zcta']].drop_duplicates().groupby(['DMA'])['zcta'].count().to_frame()
Zip_DMA_Overall_Zip.reset_index(inplace=True)
Zip_DMA_Overall_Zip.columns=['DMA','zcta_count']
Zip_DMA_Overall=pd.merge(Zip_DMA_Overall,Zip_DMA_Overall_Zip,on="DMA",how="left")

In [14]:
Zip_IRI_18_Markets.head(2)

,zcta,IRI Market,teen_population,Monther,HH_Income_0_50K,Total_Population,Total_HH
0,11742,NEW YORK,2045.0,967.51,1161.0,13132.0,4566.0
1,01005,BOSTON,1023.0,189.07,635.0,5228.0,1840.0


In [15]:
for market in IRI_Selected_18:
    df=Zip_IRI_18_Markets[Zip_IRI_18_Markets['IRI Market']==market]
    zip_df=df['zcta'].unique().tolist()
    dma_df=Zip_DMA[Zip_DMA['zcta'].isin(zip_df)].drop_duplicates()
    dma_df_in=dma_df.groupby(['DMA'])['teen_population','Monther','HH_Income_0_50K','Total_Population','Total_HH'].sum()
    dma_df_in.reset_index(inplace=True)
    dma_df_in.columns=['DMA','Teen_In_IRI','Mom_In_IRI','Lower_HH_In_IRI','Pop_all_In_IRI','HH_all_In_IRI']
    dma_df_in['Target_Audience_In_IRI']=dma_df_in['Mom_In_IRI']+dma_df_in['Teen_In_IRI']
    dma_df_in_zip=dma_df[['DMA','zcta']].drop_duplicates().groupby(['DMA'])['zcta'].count().to_frame()
    dma_df_in_zip.reset_index(inplace=True)
    dma_df_in_zip.columns=['DMA','zcta_count_In_IRI']
    dma_df_in=pd.merge(dma_df_in,dma_df_in_zip,on="DMA",how="left")
        
    
    dma_df_in=pd.merge(dma_df_in,Zip_DMA_Overall,on="DMA",how='left')
    dma_df_in['Pctg_Teen']=dma_df_in['Teen_In_IRI']/dma_df_in['teen_population']
    dma_df_in['Pctg_Mon']=dma_df_in['Mom_In_IRI']/dma_df_in['Monther']
    dma_df_in['Pctg_L_HH']=dma_df_in['Lower_HH_In_IRI']/dma_df_in['HH_Income_0_50K']
    dma_df_in['Pctg_Target_Audience']=dma_df_in['Target_Audience_In_IRI']/dma_df_in['Total_Target_Audience']
    dma_df_in['Pctg_zcta']=dma_df_in['zcta_count_In_IRI']/dma_df_in['zcta_count']
    dma_df_in['Pctg_Pop']=dma_df_in['Pop_all_In_IRI']/dma_df_in['Total_Population']
    dma_df_in['Pctg_zcta']=dma_df_in['HH_all_In_IRI']/dma_df_in['Total_HH']
    
    
    dma_df_in=dma_df_in.sort_values(['Pctg_Target_Audience','Pctg_L_HH'],ascending=[False,False])
    market_name=market.replace("/","__")
    dma_df_in.to_excel(writer,market_name,index=False)
    print(market)

LOS ANGELES
DALLAS/FT WORTH
NEW YORK
HOUSTON
WEST TEX/NEW MEXICO
PHOENIX/TUCSON
SOUTH CAROLINA
MIAMI/FT LAUDERDALE
BALTIMORE/WASHINGTON
BIRMINGHAM/MONTGOMERY
NEW ORLEANS/MOBILE
CHICAGO
MISSISSIPPI
BOSTON
ATLANTA
DETROIT
RALEIGH/GREENSBORO
DENVER


In [16]:
writer.save()